# 

In [1]:
import torch
from torchvision import transforms
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BATCH_SIZE = 64
IMG_SIZE = (128,128)
PATCH_SIZE = (8,8)
# Number of patch in one sequence
NUM_PATCH = int((IMG_SIZE[0] / PATCH_SIZE[0])**2)
# Number of patch out after feature extracting
T = int(NUM_PATCH / 4)
# Number of attention head
NUM_H = 4
# Number of Transformer Block
NUM_TR = 11

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
my_transform = transforms.Compose([
                                    transforms.Resize((IMG_SIZE)),
                                    transforms.RandAugment(),
                                    transforms.ToTensor()
])

In [5]:
def img_to_patch(x, patch_size, flatten_channels=True):
    """
    Inputs:
        x - torch.Tensor representing the image of shape [B, C, H, W]
        patch_size - Number of pixels per dimension of the patches (integer)
        flatten_channels - If True, the patches will be returned in a flattened format
                           as a feature vector instead of a image grid.
    """
    B, C, H, W = x.shape
    x = x.reshape(B, C, H//patch_size, patch_size, W//patch_size, patch_size)
    x = x.permute(0, 2, 4, 1, 3, 5) # [B, H', W', C, p_H, p_W]
    x = x.flatten(1,2)              # [B, H'*W', C, p_H, p_W]
    if flatten_channels:
        x = x.flatten(2,4)          # [B, H'*W', C*p_H*p_W]
    return x

In [6]:
class AttentionBlock(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of input and attention feature vectors
            hidden_dim - Dimensionality of hidden layer in feed-forward network
                         (usually 2-4x larger than embed_dim)
            num_heads - Number of heads to use in the Multi-Head Attention block
            dropout - Amount of dropout to apply in the feed-forward network
        """
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads,
                                          dropout=dropout)
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )
        


    def forward(self, x):
        inp_x = self.layer_norm_1(x)
        x = x + self.attn(inp_x, inp_x, inp_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x

In [7]:
class Transpose(nn.Module):
    def __init__(self, dim1, dim2):
        super().__init__()
        self.dim1 = dim1
        self.dim2 = dim2

    def forward(self, x):
        return x.transpose(self.dim1, self.dim2)


In [8]:
class VisionTransformer(nn.Module):

    def __init__(self, embed_dim, hidden_dim,n_feature, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of the input feature vectors to the Transformer
            hidden_dim - Dimensionality of the hidden layer in the feed-forward networks
                         within the Transformer
            num_channels - Number of channels of the input (3 for RGB)
            num_heads - Number of heads to use in the Multi-Head Attention block
            num_layers - Number of layers to use in the Transformer
            num_classes - Number of classes to predict
            patch_size - Number of pixels that the patches have per dimension
            num_patches - Maximum number of patches an image can have
            dropout - Amount of dropout to apply in the feed-forward network and
                      on the input encoding
        """
        super().__init__()

        self.patch_size = patch_size

        # Layers/Networks
        # self.input_layer = nn.Linear(num_channels*(patch_size**2), embed_dim)
        # self.input_layer = FeatureExtractor(n_feature,embed_dim)
        self.input_layer = nn.Sequential(
            nn.Conv3d(num_channels , n_feature*3 , kernel_size = 3 , padding=1 , stride=1),
            nn.ReLU(),
            # nn.Conv3d(n_feature , n_feature*2 , kernel_size = 3 , padding=1 , stride=1),
            # nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,1,1) , stride=(2,1,1) , padding=(0,0,0)),
            
            nn.Conv3d(n_feature*3 , n_feature*6 , 3 , padding=1 , stride=1),
            nn.ReLU(),
            # nn.Conv3d(n_feature*4 , n_feature*8 , 3 , padding=1 , stride=1),
            # nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,1,1) , stride=(2,1,1) , padding=(0,0,0)),
            nn.Conv3d(n_feature*6 , n_feature*8 , 3 , padding=1 , stride=1),
            
            
            Transpose(2,1),
            nn.Flatten(2,-1),
            nn.Linear(n_feature*8*PATCH_SIZE[0]*PATCH_SIZE[0] ,embed_dim )
            
            
        )
        self.transformer = nn.Sequential(*[AttentionBlock(embed_dim, hidden_dim, num_heads, dropout=dropout) for _ in range(num_layers)])
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
            # torch.nn.Softmax(dim=1)
        )
        self.dropout = nn.Dropout(dropout)

        # Parameters/Embeddings
        self.cls_token = nn.Parameter(torch.randn(1,1,embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1,1+int(NUM_PATCH / 4),embed_dim))


    def forward(self, x):
        # Preprocess input
        x = img_to_patch(x, self.patch_size , flatten_channels=False).transpose(1,2)
        # print(x.shape)
        B, _, _ , _ , _= x.shape
        T = int(NUM_PATCH / 4)
        x = self.input_layer(x)
        # print(x.shape)
        # Add CLS token and positional encoding
        cls_token = self.cls_token.repeat(B, 1, 1)
        x = torch.cat([cls_token, x], dim=1)
        x = x + self.pos_embedding[:,:T+1]

        # Apply Transforrmer
        x = self.dropout(x)
        x = x.transpose(0, 1)
        x = self.transformer(x)

        # Perform classification prediction
        cls = x[0]
        out = self.mlp_head(cls)
        return out

In [9]:
AmirModel = VisionTransformer(embed_dim = 256 , 
                            hidden_dim = 512,
                            n_feature=8 ,
                            num_channels = 3, 
                            num_heads = NUM_H , 
                            num_layers = NUM_TR ,
                            num_classes=4,
                            patch_size=PATCH_SIZE[0] ,
                            num_patches=NUM_PATCH , 
                            dropout=0.1 ).to(device)

**************************************************************************************

In [10]:
# Compile the model 
torch.set_float32_matmul_precision('high')
AmirModel = torch.compile(AmirModel)

In [11]:
!pwd

/app/ieee/code/Classification-Clustering/Classification


In [12]:
# AmirModel.load_state_dict(torch.load("../TrackExp/" + name + ".pth"))
AmirModel.load_state_dict(torch.load("../../../../Ablation_Study/TrackExp/BestWeight_Part_1_Epoch_3_TrainLoss_0.044724611829818246_TestLoss_0.016080412089830787TrainAcc_0.9831583453496756_Testacc_0.9951923076923077.pth"))

<All keys matched successfully>

In [13]:
train_dir = "../../../ICIP_denoised_data/Data_2/train/"
test_dir = "../../../ICIP_denoised_data/Data_2/test/"

In [54]:
from torch.utils.data import default_collate
from torchvision.transforms import v2

cutmix = v2.CutMix(num_classes=2,alpha=0.4)
mixup = v2.MixUp(num_classes=2,alpha=0.5)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])
def collate_fn(batch):
    return cutmix_or_mixup(*default_collate(batch))


In [55]:
# Setup dataloaders
from going_modular.going_modular import data_setup, engine
from helper_functions import download_data, set_seeds, plot_loss_curves

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                test_dir=test_dir,
                                                                                transform=my_transform,
                                                                                batch_size=BATCH_SIZE,
                                                                                num_workers = 8,
                                                                                collate_fn=collate_fn) # Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)


In [56]:
class_names

['1', '2']

In [57]:
next(iter(train_dataloader))[0].shape ,next(iter(train_dataloader))[1]

(torch.Size([64, 3, 128, 128]),
 tensor([[1.0000, 0.0000],
         [0.1121, 0.8879],
         [0.8879, 0.1121],
         [1.0000, 0.0000],
         [0.1121, 0.8879],
         [0.8879, 0.1121],
         [0.1121, 0.8879],
         [0.8879, 0.1121],
         [1.0000, 0.0000],
         [1.0000, 0.0000],
         [1.0000, 0.0000],
         [0.1121, 0.8879],
         [0.0000, 1.0000],
         [0.8879, 0.1121],
         [1.0000, 0.0000],
         [0.1121, 0.8879],
         [0.0000, 1.0000],
         [0.0000, 1.0000],
         [0.8879, 0.1121],
         [0.1121, 0.8879],
         [0.0000, 1.0000],
         [0.8879, 0.1121],
         [1.0000, 0.0000],
         [1.0000, 0.0000],
         [1.0000, 0.0000],
         [1.0000, 0.0000],
         [1.0000, 0.0000],
         [0.1121, 0.8879],
         [0.0000, 1.0000],
         [0.8879, 0.1121],
         [0.1121, 0.8879],
         [0.8879, 0.1121],
         [0.1121, 0.8879],
         [0.0000, 1.0000],
         [0.0000, 1.0000],
         [0.8879, 0.112

In [58]:
print(f"train_dir: {train_dir}")
print(f"train_dir: {test_dir}")
print(f"len train_dataloader: {len(train_dataloader)}")
print(f"len test_dataloader: {len(test_dataloader)}")
print(f"device: {device}")

train_dir: ../../../ICIP_denoised_data/Data_2/train/
train_dir: ../../../ICIP_denoised_data/Data_2/test/
len train_dataloader: 115
len test_dataloader: 29
device: cuda


In [59]:
# Create optimizer and loss function
optimizer = torch.optim.SGD(AmirModel.parameters() , 0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [60]:
AmirModel = nn.Sequential(AmirModel,nn.Linear(4,2)).to(device)

In [61]:
from going_modular.going_modular import engine
part = "ICIP_final_denoise"
# Train the classifier head of the pretrained ViT feature extractor model
set_seeds()
x = engine.train(model=AmirModel,
                                      train_dataloader=train_dataloader,
                                      test_dataloader=test_dataloader,
                                      optimizer=optimizer,
                                      loss_fn=loss_fn,
                                      epochs=50,
                                      Pivot=65,
                                      Part=part,
                                      save_weights=True,
                                      device=device)

  0%|                                                                                                                                | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
  2%|██▎                                                                                                                   | 1/50 [01:16<1:02:04, 76.01s/it]

Epoch: 1 | train_loss: 0.7782 | train_acc: 0.5873 | test_loss: 0.7623 | test_acc: 0.5361


  4%|████▊                                                                                                                   | 2/50 [01:40<36:25, 45.53s/it]

Epoch: 2 | train_loss: 0.6646 | train_acc: 0.6270 | test_loss: 0.7225 | test_acc: 0.5857


  6%|███████▏                                                                                                                | 3/50 [02:04<28:08, 35.92s/it]

...

Epoch: 48 | train_loss: 0.5495 | train_acc: 0.7575 | test_loss: 0.6692 | test_acc: 0.6460


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 49/50 [21:01<00:26, 26.96s/it]

Epoch: 49 | train_loss: 0.5427 | train_acc: 0.7584 | test_loss: 0.6918 | test_acc: 0.6336


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [21:37<00:00, 25.95s/it]

Epoch: 50 | train_loss: 0.5312 | train_acc: 0.7735 | test_loss: 0.7358 | test_acc: 0.6331


## Save model

In [ ]:
# # Save the model
from going_modular.going_modular import utils

utils.save_model(model=AmirModel,
                 target_dir="./TrackExp",
                 model_name=name2 + ".pth")

## Load model weights

In [22]:
name2 = "TrackExp/BestWeight_Part_19_Epoch_2_TrainLoss_0.01639417985006382_TestLoss_0.027976555515579093TrainAcc_0.9941451149425288_Testacc_0.9951171875.pth"
AmirModel.load_state_dict(torch.load(name2))

<All keys matched successfully>